# Mergem model mapping
This code implements methods for mapping model contents into the MetaNetX namespace using the mergem library.

# 1. Installing required libraries

In [1]:
import mergem
import cobra
from collections import defaultdict
import pandas as pd
import re
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, leaves_list, dendrogram
from scipy.spatial.distance import pdist
import numpy as np

# 2. Function definitions

## 2.1 Functions for model inspection

In [2]:
def print_rxns(model,n):
    """
    Prints total number of reactions, and first n reactions of a model.
    """
    # Print number of reactions
    num_reactions = len(model.reactions)
    print(f"Number of reactions in the model: {num_reactions}")

    # Print n first reactions
    print("\nFirst n reactions in the model:")
    for reaction in model.reactions[:n]:
        print(reaction.id,': ', reaction.reaction)


def print_mets(model,n):
    """
    Prints total number of metabolites, and first n metabolites of a model.
    """
    # Print number of metabolites
    num_metabolites = len(model.metabolites)
    print(f"\nNumber of metabolites in the model: {num_metabolites}")

    # Print n first metabolites
    print("\nFirst n metabolites in the model:")
    for metabolite in model.metabolites[:n]:
        print(metabolite.id,': ', metabolite.formula)


def print_compartments(model):
    """
    Prints compartments of a model.
    """
    # Print compartments
    print("\nCompartments in the model:")
    for compartment_id, compartment_name in model.compartments.items():
        print(f"{compartment_id}: {compartment_name}")


def print_transports(model,n):
    """
    Prints total number of transports, and first n transports of a model
    """
    # Print number of transports
    transport_rxns = []
    for rxn in model.reactions:
        compartments = {met.compartment for met in rxn.metabolites}
        if len(compartments) > 1:
            transport_rxns.append(rxn)
    print(f"\nNumber of transport reactions in the model: {len(transport_rxns)}")

    # Print n first transports
    print("\nFirst n transport reactions in the model:")
    for reaction in transport_rxns[:n]:
        print(reaction.id, ': ', reaction.reaction)


def print_boundaries(model,n):
    """
    Prints total number of boundaries, and first n boundaries of a model
    """
    # Print number of boundaries
    boundary_rxns = [rxn for rxn in model.reactions if rxn.boundary]
    print(f"\nNumber of boundary reactions in the model: {len(boundary_rxns)}")
    
    # Print n first boundaries
    print("\nFirst n boundary reactions in the model:")
    for rxn in boundary_rxns[:n]:
        print(rxn.id, ': ', rxn.reaction)


def print_shared_core_rxns(model,n):
    """
    Prints total number of reactions sharing prefix (until the first underscore), and first n of such reactions. 
    These reactions are typically biochemically equivalent but ocurring in different compartments.
    """
    # Number of duplicated reactions in the model
    duplicated_count = 0
    duplicated_reactions = defaultdict(list)
    
    # Group reactions by core ID (everything before the first underscore)
    for reaction in model.reactions:
        core_id = reaction.id.split('_')[0]
        duplicated_reactions[core_id].append(reaction)

    # Filter out core IDs with more than one reaction
    duplicate_instances = {core_id: reactions for core_id, reactions in duplicated_reactions.items() if len(reactions) > 1}
    
    # Count the duplicates
    duplicated_count = sum(len(reactions) - 1 for reactions in duplicate_instances.values())
    print(f"\nNumber of putatively duplicated reactions, i.e. sharing core ID (everything before the first underscore): {duplicated_count}")
    
    # Print first n duplicated reaction groups
    print("\nFirst n putatively duplicated reactions:")
    i = 0
    for core_id, reactions in duplicate_instances.items():
        if i >= n:
            break
        else:
            print(f"{core_id}: {[rxn.id for rxn in reactions]}")
            i += 1
    print('\n')


def summarize_model_info(model, n):
    """
    Runs all previously defined printing functions
    """
    
    #Print reactions
    print_rxns(model,n)

    #Print metabolites
    print_mets(model,n)

    #Print compartments
    print_compartments(model)

    #Print transports
    print_transports(model,n)
    
    #Print boundaries
    print_boundaries(model,n)
    
    #Print rxns with shared core
    print_shared_core_rxns(model,n)

## 2.2 Functions for model mapping

In [3]:
def is_transport(reaction):
    """
    Decides whether a reaction is transport/exchange or not
    """
    compartments = {met.compartment for met in reaction.metabolites}
    if len(compartments) > 1:
        return 1
    elif reaction.boundary:
        return 1
    else:
        return 0

def extract_core_id(s):
    """
    Extracts the 'core' id from an id. For models already mapped to MetaNetX via mergem, the core is 
    the substring starting with 'MNX' until a '~' or '_' is reached (or the original string otherwise).
    If a match is found, it is returned. If no match is found, the original string is returned.
    """
    match = re.search(r'MNX(?:[^\_~]*)?', s)
    return match.group(0) if match else s


def export_mergem_mapping(model, model_name, output_mapping_path):
    """
    Creates and exports tables summarizing metabolite and reaction ID mappings using mergem 
    """
    
    original_rxn_ids = [rxn.id for rxn in model.reactions] # All reactions in the original model
    original_met_ids = [met.id for met in model.metabolites] # All metabolites in the original model
    
    mapped_model = mergem.translate(model, 'metanetx') # Mapped model
    
    metanetx_rxn_ids = [rxn.id for rxn in mapped_model.reactions] # All reactions in the mapped model
    transport_rxn_class = [is_transport(rxn) for rxn in mapped_model.reactions] # Binary vector encoding transport reactions
    core_metanetx_rxn_ids = [extract_core_id(rxn.id) if not is_transport(rxn) \
                             else 'Transport removed' for rxn in mapped_model.reactions]  # Deduplicated reaction IDs (or tagged as transport)
    
    metanetx_met_ids = [met.id for met in mapped_model.metabolites] #All metabolites in the mapped model
    core_metanetx_met_ids = [extract_core_id(met.id) for met in mapped_model.metabolites] # Deduplicated metabolite IDs

    # Create and export mapping tables
    rxn_df = pd.DataFrame({'Original ID':original_rxn_ids,'Raw MetaNetX ID':metanetx_rxn_ids,'transport':transport_rxn_class,\
                           'Final ID':core_metanetx_rxn_ids}) 
    
    met_df = pd.DataFrame({'Original ID':original_met_ids,'Raw MetaNetX ID':metanetx_met_ids,'Final ID':core_metanetx_met_ids})
    rxn_df.to_csv(output_mapping_path + '/mergem_reaction_mapping/' + model_name + '.csv', index=False) 
    met_df.to_csv(output_mapping_path + '/mergem_metabolite_mapping/' + model_name + '.csv', index=False) 

# 3. Mappings

## 3.0 Path definitions

In [4]:
input_model_path = 'models' # Folder with generated models
output_mapping_path = 'mappings' # Folder where mapping results will be saved

## 3.1 AuReMe

### 3.1.1 Model importing

In [5]:
au_CHO = cobra.io.read_sbml_model(input_model_path + '/' + 'AuReMe/CHO.xml')
au_A_aegypti = cobra.io.read_sbml_model(input_model_path + '/' + 'AuReMe/A_aegypti.xml')
au_E_siliculosus = cobra.io.read_sbml_model(input_model_path + '/' + 'AuReMe/E_siliculosus.xml')

Loading SBML model without fbc:strict="true"
Loading SBML with fbc-v1 (models should be encoded using fbc-v2)
No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized
Loading SBML model without fbc:strict="true"
Loading SBML with fbc-v1 (models should be encoded using fbc-v2)
No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized
Loading SBML model without fbc:strict="true"
Loading SBML with fbc-v1 (models should be encoded using fbc-v2)
No objective in listOfObjectives
No objective coefficients in model. Unclear what should be optimized


### 3.1.2 Mapping to MetaNetX

In [11]:
#Export AureMe CHO mergem results
export_mergem_mapping(au_CHO, 'au_CHO', output_mapping_path)

#Export AureMe A. aegypti mergem results
export_mergem_mapping(au_A_aegypti, 'au_A_aegypti', output_mapping_path)

#Export AureMe E. siliculosus mergem results
export_mergem_mapping(au_E_siliculosus, 'au_E_siliculosus', output_mapping_path)

## 3.2 carveMe

### 3.2.1 Model importing

In [12]:
cv_CHO = cobra.io.read_sbml_model(input_model_path + '/' + 'carveMe/CHO.xml')
cv_A_aegypti = cobra.io.read_sbml_model(input_model_path + '/' + 'carveMe/A_aegypti.xml')
cv_E_siliculosus = cobra.io.read_sbml_model(input_model_path + '/' + 'carveMe/E_siliculosus.xml')

### 3.2.2 Mapping to MetaNetX

In [13]:
#Export CHO mergem results
export_mergem_mapping(cv_CHO, 'cv_CHO', output_mapping_path)

#Export A. aegypti mergem results
export_mergem_mapping(cv_A_aegypti, 'cv_A_aegypti', output_mapping_path)

#Export E. siliculosus mergem results
export_mergem_mapping(cv_E_siliculosus, 'cv_E_siliculosus', output_mapping_path)

## 3.3 merlin 

### 3.3.1 Model importing

In [14]:
me_CHO = cobra.io.read_sbml_model(input_model_path + '/' + 'merlin/CHO.xml')
me_A_aegypti = cobra.io.read_sbml_model(input_model_path + '/' + 'merlin/A_aegypti.xml')
me_E_siliculosus = cobra.io.read_sbml_model(input_model_path + '/' + 'merlin/E_siliculosus.xml')

10029 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
7159 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
2880 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id


### 3.3.2 Mapping to MetaNetX

In [15]:
#Export CHO mergem results
export_mergem_mapping(me_CHO, 'me_CHO', output_mapping_path)

#Export A. aegypti mergem results
export_mergem_mapping(me_A_aegypti, 'me_A_aegypti', output_mapping_path)

#Export E. siliculosus mergem results
export_mergem_mapping(me_E_siliculosus, 'me_E_siliculosus', output_mapping_path)

## 3.4 modelSEED

### 3.4.1 Model importing

In [ ]:
ms_CHO = cobra.io.read_sbml_model(input_model_path + '/' + 'modelseed/CHO.xml')
ms_A_aegypti = cobra.io.read_sbml_model(input_model_path + '/' + 'modelseed/A_aegypti.xml')
ms_E_siliculosus = cobra.io.read_sbml_model(input_model_path + '/' + 'modelseed/E_siliculosus.xml')

Model does not contain SBML fbc package information.
SBML package 'layout' not supported by cobrapy, information is not parsed
SBML package 'render' not supported by cobrapy, information is not parsed
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd00001_c0 "H2O_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd00009_c0 "Phosphate_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd00012_c0 "PPi_c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd00067_c0 "H_plus__c0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd00001_d0 "H2O_d0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd00009_d0 "Phosphate_d0">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_cpd00012_d0 "PPi_d0">
Use of the species c

### 3.4.2 Mapping to MetaNetX

In [ ]:
#Export CHO mergem results
export_mergem_mapping(ms_CHO, 'ms_CHO', output_mapping_path)

#Export A. aegypti mergem results
export_mergem_mapping(ms_A_aegypti, 'ms_A_aegypti', output_mapping_path)

#Export E. siliculosus mergem results
export_mergem_mapping(ms_E_siliculosus, 'ms_E_siliculosus', output_mapping_path)

## 3.5 Pathway Tools

### 3.5.1 Model importing

In [ ]:
pt_CHO = cobra.io.read_sbml_model(input_model_path + '/' + 'pathway_tools/CHO.xml')
pt_A_aegypti = cobra.io.read_sbml_model(input_model_path + '/' + 'pathway_tools/A_aegypti.xml')
pt_E_siliculosus = cobra.io.read_sbml_model(input_model_path + '/' + 'pathway_tools/E_siliculosus.xml')

### 3.5.2 Mapping to MetaNetX

In [ ]:
#Export CHO mergem results
export_mergem_mapping(pt_CHO, 'pt_CHO', output_mapping_path)

#Export A. aegypti mergem results
export_mergem_mapping(pt_A_aegypti, 'pt_A_aegypti', output_mapping_path)

#Export E. siliculosus mergem results
export_mergem_mapping(pt_E_siliculosus, 'pt_E_siliculosus', output_mapping_path)

## 3.6 Raven

### 3.6.1 Model importing

In [ ]:
r_CHO = cobra.io.read_sbml_model(input_model_path + '/' + 'raven/CHO.xml')
r_A_aegypti = cobra.io.read_sbml_model(input_model_path + '/' + 'raven/A_aegypti.xml')
r_E_siliculosus = cobra.io.read_sbml_model(input_model_path + '/' + 'raven/E_siliculosus.xml')

### 3.6.2 Mapping to MetaNetX

In [ ]:
#Export CHO mergem results
export_mergem_mapping(r_CHO, 'r_CHO', output_mapping_path)

#Export A. aegypti mergem results
export_mergem_mapping(r_A_aegypti, 'r_A_aegypti', output_mapping_path)

#Export E. siliculosus mergem results
export_mergem_mapping(r_E_siliculosus, 'r_E_siliculosus', output_mapping_path)

## 3.7 Raven homo

### 3.7.1 Model importing

In [ ]:
rh_CHO = cobra.io.read_sbml_model(input_model_path + '/' + 'raven_homo/CHO.xml')
rh_A_aegypti = cobra.io.read_sbml_model(input_model_path + '/' + 'raven_homo/A_aegypti.xml')
rh_E_siliculosus = cobra.io.read_sbml_model(input_model_path + '/' + 'raven_homo/E_siliculosus.xml')

### 3.7.2 Mapping to MetaNetX

In [ ]:
#Export CHO mergem results
export_mergem_mapping(rh_CHO, 'rh_CHO', output_mapping_path)

#Export A. aegypti mergem results
export_mergem_mapping(rh_A_aegypti, 'rh_A_aegypti', output_mapping_path)

#Export E. siliculosus mergem results
export_mergem_mapping(rh_E_siliculosus, 'rh_E_siliculosus', output_mapping_path)

## 3.8 Raven combined

### 3.8.1 Model importing

In [ ]:
rc_CHO = cobra.io.read_sbml_model(input_model_path + '/' + 'raven_comb/CHO.xml')
rc_A_aegypti = cobra.io.read_sbml_model(input_model_path + '/' + 'raven_comb/A_aegypti.xml')
rc_E_siliculosus = cobra.io.read_sbml_model(input_model_path + '/' + 'raven_comb/E_siliculosus.xml')

### 3.8.2 Mapping to MetaNetX with mergem

In [ ]:
#Export CHO mergem results
export_mergem_mapping(rc_CHO, 'rc_CHO', output_mapping_path)

#Export A. aegypti mergem results
export_mergem_mapping(rc_A_aegypti, 'rc_A_aegypti', output_mapping_path)

#Export E. siliculosus mergem results
export_mergem_mapping(rc_E_siliculosus, 'rc_E_siliculosus', output_mapping_path)

## 3.9 Reconstructor

### 3.9.1 Model importing

In [ ]:
rec_CHO = cobra.io.read_sbml_model(input_model_path + '/' + 'reconstructor/CHO.sbml')
rec_A_aegypti = cobra.io.read_sbml_model(input_model_path + '/' + 'reconstructor/A_aegypti.sbml')
rec_E_siliculosus = cobra.io.read_sbml_model(input_model_path + '/' + 'reconstructor/E_siliculosus.sbml')

### 3.9.2 Mapping to MetaNetX

In [ ]:
#Export CHO mergem results
export_mergem_mapping(rec_CHO, 'rec_CHO', output_mapping_path)

#Export A. aegypti mergem results
export_mergem_mapping(rec_A_aegypti, 'rec_A_aegypti', output_mapping_path)

#Export E. siliculosus mergem results
export_mergem_mapping(rec_E_siliculosus, 'rec_E_siliculosus', output_mapping_path)

## 3.10 AuReMe (mode 2)

### 3.10.1 Model importing

In [ ]:
aedes_PWT = cobra.io.read_sbml_model('AuReMe_test/models_annotation_only/Aedes_PWT.sbml')
CHO_PWT = cobra.io.read_sbml_model('AuReMe_test/models_annotation_only/CHO_PWT.sbml')
Ectocarpus_PWT = cobra.io.read_sbml_model('AuReMe_test/models_annotation_only/Ectocarpus_PWT.sbml')

### 3.10.2 Mapping to MetaNetX

In [ ]:
#Export A. aegypti mergem results
export_mergem_mapping(aedes_PWT, 'au2_A_aegypti', output_mapping_path)

#Export CHO mergem results
export_mergem_mapping(CHO_PWT, 'au2_CHO', output_mapping_path)

#Export E. siliculosus mergem results
export_mergem_mapping(Ectocarpus_PWT, 'au2_E_siliculosus', output_mapping_path)

## 3.11 AuReMe (mode 3)

### 3.11.1 Model importing

In [ ]:
Aedes_Recon = cobra.io.read_sbml_model('AuReMe_test/models_merge/Aedes_Recon.sbml')
CHO_Recon = cobra.io.read_sbml_model('AuReMe_test/models_merge/CHO_Recon.sbml')
Ecto_Saccharina_Aureme = cobra.io.read_sbml_model('AuReMe_test/models_merge/Ecto_Saccharina_Aureme.sbml')

### 3.11.2 Mapping to MetaNetX

In [ ]:
#Export A. aegypti mergem results
export_mergem_mapping(Aedes_Recon, 'au3_A_aegypti', output_mapping_path)

#Export CHO mergem results
export_mergem_mapping(CHO_Recon, 'au3_CHO', output_mapping_path)

#Export E. siliculosus mergem results
export_mergem_mapping(Ecto_Saccharina_Aureme, 'au3_E_siliculosus', output_mapping_path)

## 3.12 AuReMe (mode 4)

### 3.12.1 Model importing

In [ ]:
Aedes_orthofinder = cobra.io.read_sbml_model('AuReMe_test/models_orthology_only/Aedes_orthofinder.sbml')
CHO_orthofinder = cobra.io.read_sbml_model('AuReMe_test/models_orthology_only/CHO_orthofinder.sbml')
Ectocarpus_orthofinder = cobra.io.read_sbml_model('AuReMe_test/models_orthology_only/Ectocarpus_orthofinder.sbml')

### 3.12.2 Mapping to MetaNetX

In [ ]:
#Export A. aegypti mergem results
export_mergem_mapping(Aedes_orthofinder, 'au4_A_aegypti', output_mapping_path)

#Export CHO mergem results
export_mergem_mapping(CHO_orthofinder, 'au4_CHO', output_mapping_path)

#Export E. siliculosus mergem results
export_mergem_mapping(Ectocarpus_orthofinder, 'au4_E_siliculosus', output_mapping_path)

## 3.13 AuReMe (mode 5)

### 3.13.1 Model importing

In [ ]:
CHO_Recon_orthofinder = cobra.io.read_sbml_model('AuReMe_test/models_orthology_only/CHO_Recon_orthofinder.sbml')
Ectocarpus_Saccharina_orthofinder = cobra.io.read_sbml_model('AuReMe_test/models_orthology_only/Ectocarpus_Saccharina_orthofinder.sbml')

### 3.13.2 Mapping to MetaNetX

In [ ]:
#Export E. siliculosus mergem results
export_mergem_mapping(CHO_Recon_orthofinder, 'au5_CHO', output_mapping_path)

#Export E. siliculosus mergem results
export_mergem_mapping(Ectocarpus_Saccharina_orthofinder, 'au5_E_siliculosus', output_mapping_path)

## 3.14 Reference

### 3.14.1 Model importing

In [5]:
ref_CHO = cobra.io.read_sbml_model(input_model_path + '/' + 'reference/CHO.xml')
ref_A_aegypti = cobra.io.read_sbml_model(input_model_path + '/' + 'reference/A_aegypti.xml')
ref_E_siliculosus = cobra.io.read_sbml_model(input_model_path + '/' + 'reference/E_siliculosus.xml')

Loading SBML model without fbc:strict="true"
Loading SBML with fbc-v1 (models should be encoded using fbc-v2)
Adding exchange reaction EX_MG+2_C-BOUNDARY with default bounds for boundary metabolite: MG+2_C-BOUNDARY.
Adding exchange reaction EX_NITRATE_C-BOUNDARY with default bounds for boundary metabolite: NITRATE_C-BOUNDARY.
Adding exchange reaction EX_BIOTIN_C-BOUNDARY with default bounds for boundary metabolite: BIOTIN_C-BOUNDARY.
Adding exchange reaction EX_CARBON-DIOXIDE_C-BOUNDARY with default bounds for boundary metabolite: CARBON-DIOXIDE_C-BOUNDARY.
Adding exchange reaction EX_CU+_C-BOUNDARY with default bounds for boundary metabolite: CU+_C-BOUNDARY.
Adding exchange reaction EX_CA+2_C-BOUNDARY with default bounds for boundary metabolite: CA+2_C-BOUNDARY.
Adding exchange reaction EX_CL-_C-BOUNDARY with default bounds for boundary metabolite: CL-_C-BOUNDARY.
Adding exchange reaction EX_ADENOSYLCOBALAMIN_C-BOUNDARY with default bounds for boundary metabolite: ADENOSYLCOBALAMIN_C-

### 3.14.2 Mapping to MetaNetX

In [6]:
#Export A. aegypti mergem results
export_mergem_mapping(ref_A_aegypti, 'ref_A_aegypti', output_mapping_path)

#Export CHO mergem results
export_mergem_mapping(ref_CHO, 'ref_CHO', output_mapping_path)

#Export E. siliculosus mergem results
export_mergem_mapping(ref_E_siliculosus, 'ref_E_siliculosus', output_mapping_path)